In [ ]:
from google.colab import drive
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Clone NHÁNH convnext-v2-dev
if not os.path.exists('/content/FR_Photometric_Stereo'):
    !git clone -b convnext-v2-dev https://github.com/minh4923/FR_Photometric_Stereo.git

# 3. Di chuyển vào thư mục dự án
%cd /content/FR_Photometric_Stereo

# 4. Cài đặt thư viện
!pip install -q albumentations==1.3.1 timm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/FR_Photometric_Stereo


In [ ]:
%cd /content/FR_Photometric_Stereo
import warnings
warnings.filterwarnings("ignore")
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A
import os
import pandas as pd

from going_modular.dataloader.multitask import create_multitask_datafetcher
from going_modular.model.MTLFaceRecognition import MTLFaceRecognition
from going_modular.loss.MultiTaskLoss import MultiTaskLoss
from going_modular.train_eval.train import fit
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.ExperimentManager import ExperimentManager

# --- CẤU HÌNH ---
device = "cuda" if torch.cuda.is_available() else "cpu"
seed = 42
torch.manual_seed(seed)

# Đặt tên thí nghiệm (Folder sẽ được tạo theo tên này)
EXPERIMENT_NAME = "Single_ALBEDO_Shuffle"

CONFIGURATION = {
    'note': EXPERIMENT_NAME,
    'dataset_dir': '/content/drive/MyDrive/Photometric_DB_Full/',
    'type': 'albedo',

    'device': device,
    'epochs': 120,
    'batch_size': 32,
    'image_size': 112,
    'base_lr': 1e-4,
    'backbone': 'miresnet18', # Có thể đổi thành miresnet50
    'num_classes': None,

    # Trọng số đã tối ưu (Giảm task phụ để tập trung vào ID)
    'loss_gender_weight': 1.0,      'loss_da_gender_weight': 0.1,
    'loss_emotion_weight': 0.5,     'loss_da_emotion_weight': 0.1,
    'loss_pose_weight': 1.0,        'loss_da_pose_weight': 0.1,
    'loss_spectacles_weight': 0.5,  'loss_da_spectacles_weight': 0.1,
    'loss_facial_hair_weight': 0.5, 'loss_da_facial_hair_weight': 0.1,
}

# --- 1. KHỞI TẠO MANAGER ---
manager = ExperimentManager(CONFIGURATION)
manager.log_text(f"TRAINING MODE: {CONFIGURATION['type']} | Backbone: {CONFIGURATION['backbone']}")

# --- 2. DATASET ---
train_csv_path = os.path.join(CONFIGURATION['dataset_dir'], 'dataset/train_split.csv')
if not os.path.exists(train_csv_path): train_csv_path = os.path.join(CONFIGURATION['dataset_dir'], 'train_split.csv')

# Load CSV để lấy num_classes chính xác
df_temp = pd.read_csv(train_csv_path)
CONFIGURATION['num_classes'] = df_temp['id'].max() + 1 # An toàn nhất: Max ID + 1 để tránh lỗi index out of bounds
manager.log_text(f"Max ID in CSV: {df_temp['id'].max()} -> Set num_classes: {CONFIGURATION['num_classes']}")

train_transform = A.Compose([
    RandomResizedCropRect(CONFIGURATION['image_size']),
    GaussianNoise(p=0.2), A.HorizontalFlip(p=0.5)
], additional_targets={'albedo': 'image', 'depthmap': 'image'})

test_transform = A.Compose([A.Resize(CONFIGURATION['image_size'], CONFIGURATION['image_size'])],
                           additional_targets={'albedo': 'image', 'depthmap': 'image'})

train_dl, test_dl, _ = create_multitask_datafetcher(CONFIGURATION, train_transform, test_transform)

# --- 3. MODEL & LOSS ---
model = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])
# Lưu ý: Truyền đúng config vào Loss để nó nhận weights mới
criterion = MultiTaskLoss(train_csv_path, CONFIGURATION)
optimizer = Adam(model.parameters(), lr=CONFIGURATION['base_lr'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2)

# --- 4. CHECKPOINT SAVER ---
# Lưu vào manager.ckpt_dir (experiments/.../checkpoints)
ckpt_saver = ModelCheckpoint(
    output_dir=manager.ckpt_dir,
    mode='max',
    best_metric_name='auc_id_cosine' # Model tốt nhất là model có AUC Cosine cao nhất
)

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc'], patience=1500, mode='max', verbose=0,
    save_dir=manager.ckpt_dir, start_from_epoch=0
)

# --- 5. RUN ---
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    train_dataloader=train_dl,
    test_dataloader=test_dl,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    early_stopping=early_stopping,
    model_checkpoint=ckpt_saver,
    existing_manager=manager
)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
├─────────────────────┼───────────┼─────────────────────┤
│ loss_da_pose        │  0.655968 │ 0.6701568365097046  │
├─────────────────────┼───────────┼─────────────────────┤
│ loss_facial_hair    │  0.355797 │ 0.526133249203364   │
├─────────────────────┼───────────┼─────────────────────┤
│ loss_da_facial_hair │  0.606846 │ 0.6279801328976949  │
├─────────────────────┼───────────┼─────────────────────┤
│ loss_spectacles     │  0.193737 │ 0.25682522190941703 │
├─────────────────────┼───────────┼─────────────────────┤
│ loss_da_spectacles  │  0.686349 │ 0.7243497769037882  │
├─────────────────────┼───────────┼─────────────────────┤
│ auc_gender          │  0.990254 │ 0.965400061785604   │
├─────────────────────┼───────────┼─────────────────────┤
│ auc_spectacles      │  0.992186 │ 0.9612967914438504  │
├─────────────────────┼───────────┼─────────────────────┤
│ auc_facial_hair     │  0.917717 │ 0.8051579252390612  │
├──────────────